
# MindsAI EEG: Real‑Time Filtering + Metrics — Documentation Notebook

**Purpose:** A single, copy‑pasteable notebook that documents the configuration, math, and code patterns used in your real‑time EEG filtering app. It includes:

- A **Quick‑Start / README** with top‑level configs (license key, BrainFlow vs manual sampling rate, noise toggles, thresholds).
- Copy‑pasteable **code snippets** for license initialization, sampling rate, optional bandpass, detrend, noise injection, and console summaries.
- Clear **math (LaTeX)** for SNR variants, peak suppression, drift, and variance reduction — with notes on single‑ vs multi‑channel behavior.
- A **Full Script (reference)** cell using your current code (kept as a non‑executing reference here).

> **Note:** The cells are meant to be read and the code is safe to copy back into your main app. This notebook does **not** attempt to open hardware connections by default.



## Quick‑Start / README

### 1) License initialization
```python
import mindsai_filter_python

# Replace with your key
mindsai_filter_python.initialize_mindsai_license('MINDS-TEST-001')
print(mindsai_filter_python.get_mindsai_license_message())
```



### 2) Core configuration (edit in one place)
```python
# === CONFIG ===
filterHyperparameter = 1e-25
channel_idx = 4            # EEG channel to display (0-based within EEG subset)
window_seconds = 1

# Noise switches
ENABLE_NOISE_HIGHLIGHT = True
INJECT_BURST = False        # Orange  (only applies to channel_idx)
INJECT_FLAT  = False        # Red     (only applies to channel_idx)
INJECT_SINE  = False        #         (applies to all channels)
INJECT_WHITE = False        #         (applies to all channels)

# Optional pre-filter (bandpass) BEFORE MindsAI
ENABLE_BANDPASS = True
BP_LOW_HZ       = 1.0
BP_HIGH_HZ      = 40.0
BP_ORDER        = 4
from brainflow.data_filter import FilterTypes
BP_TYPE         = FilterTypes.BUTTERWORTH   # Use ZERO_PHASE for non-causal phase-neutral BP

# Console description thresholds
ARTIFACT_SUPPRESSION_THRESH = 20.0   # % peak reduction to call it "Artifact Suppression"
DRIFT_THRESH_UV             = 5.0    # |mean| or |median| shift (μV) to call it "Drift Correction"
VARIANCE_SMOOTHING_THRESH   = 5.0    # % variance reduction to call it "Smoothing Effect"

# SNR method: "variance_ratio" | "power_ratio" | "amplitude_ratio"
SNR_METHOD = "amplitude_ratio"
```



### 3) BrainFlow vs Manual sampling rate
```python
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

USE_BRAINFLOW = True
USE_SYNTHETIC = False
params = BrainFlowInputParams()

board_id = BoardIds.SYNTHETIC_BOARD.value if USE_SYNTHETIC else BoardIds.CYTON_DAISY_BOARD.value

MANUAL_FS = 125  # used only if USE_BRAINFLOW = False

if USE_BRAINFLOW:
    fs = BoardShim.get_sampling_rate(board_id)
else:
    fs = MANUAL_FS

if not USE_SYNTHETIC:
    params.serial_port = "COM3"
    UNIT_SCALE_IN = 1e-6  # µV → V for MAI filter (update if your board is already V)
else:
    UNIT_SCALE_IN = 1.0   # synthetic already in V
```



### 4) Time axis on plots (UTC ms)
Use Matplotlib's date axis so you see **true time**, not sample indices.



## Math (LaTeX)

### Signal‑to‑Noise Ratio (SNR)

We treat the **filtered** signal \( s[n] \) and the **removed noise estimate** \( e[n] = x[n] - s[n] \). Then define SNR in three variants:

- **Power ratio**
\[
\mathrm{SNR_{dB}} = 10\log_{10}\frac{\mathbb{E}[s^2]}{\mathbb{E}[e^2]}
\]

- **Variance ratio**
\[
\mathrm{SNR_{dB}} = 10\log_{10}\frac{\mathrm{Var}(s)}{\mathrm{Var}(e)}
\]

- **Amplitude ratio**
\[
\mathrm{SNR_{dB}} = 20\log_{10}\frac{\mathbb{E}[|s|]}{\mathbb{E}[|e|]}
\]

We also print an intuitive fraction:
\[
\text{signal\_power\_fraction} = \frac{\mathrm{SNR_{lin}}}{1+\mathrm{SNR_{lin}}},
\quad \mathrm{where}\quad \mathrm{SNR_{lin}} = 10^{\mathrm{SNR_{dB}}/10}.
\]

### Peak suppression (artifact attenuation)

Define
\[
P_\text{before}=\max_n |x[n]|,\quad P_\text{after}=\max_n |s[n]|,\quad
\Delta P = P_\text{before}-P_\text{after},\quad
\%\Delta P=100\cdot\frac{\Delta P}{P_\text{before}}.
\]

### Baseline drift

We monitor the mean and median shifts:
\[
\Delta\mu=\mu_s-\mu_x,\qquad
\Delta\tilde{x}=\mathrm{median}(s)-\mathrm{median}(x).
\]

If either exceeds a threshold (e.g., \(5\,\mu\mathrm{V}\)), we tag **Drift Correction**.

### Variance reduction (smoothing)

\[
\%\Delta\sigma^2 = 100\cdot\frac{\mathrm{Var}(x)-\mathrm{Var}(s)}{\mathrm{Var}(x)}.
\]

If this exceeds a threshold (e.g., \(5\%\)), we tag **Smoothing Effect**.

### Channel model notes

- **Single‑electrode artifacts** (burst, flatline) are injected **only on the displayed channel** for realism.
- **Global noise** (white, line) may affect all channels.
- Channel averaging uses only **EEG channels** (from `BoardShim.get_eeg_channels(board_id)`).


## Reusable Code Snippets

In [ ]:

# Noise injection (single-channel function)
import numpy as np

def inject_noise(signal, fs, white=True, sine=True, burst=True, flat=True):
    """Synthetic noise model.
    - white: Gaussian across whole chunk
    - sine:  60 Hz line interference across whole chunk
    - burst: 2 transient spikes (e.g. muscular/jitter), 50 samples each
    - flat:  1 short flat segment (e.g. clipping), 25 samples
    Returns: noisy, burst_ranges, flat_ranges (ranges are chunk-relative indices)
    """
    t = np.arange(len(signal)) / fs
    noisy = signal.copy()
    burst_ranges = []
    flatline_ranges = []

    if white:
        noisy += np.random.normal(0, 5, size=signal.shape)
    if sine:
        noisy += 10 * np.sin(2 * np.pi * 60 * t)
    if burst:
        for _ in range(2):
            s = np.random.randint(0, max(1, len(signal) - 50))
            noisy[s:s+50] += np.random.normal(40, 10, 50)
            burst_ranges.append((s, s + 50))
    if flat:
        s = np.random.randint(0, max(1, len(signal) - 25))
        noisy[s:s+25] = 0
        flatline_ranges.append((s, s + 25))

    return noisy, burst_ranges, flatline_ranges


In [ ]:

# Metrics: SNR and filter impact
import numpy as np

def calculate_snr(signal, noise, method="variance_ratio"):
    if method == "power_ratio":
        s = np.mean(signal**2)
        n = np.mean(noise**2)
    elif method == "variance_ratio":
        s = np.var(signal)
        n = np.var(noise)
    elif method == "amplitude_ratio":
        s = np.mean(np.abs(signal))
        n = np.mean(np.abs(noise))
    else:
        raise ValueError(f"Unknown SNR method: {method}")
    if n <= 0:
        return float("inf")
    return 10*np.log10(s/n)

def calculate_filter_impact(raw_signal, filtered_signal):
    peak_before = float(np.max(np.abs(raw_signal)))
    peak_after  = float(np.max(np.abs(filtered_signal)))
    peak_reduction = peak_before - peak_after
    mean_shift   = float(np.mean(filtered_signal) - np.mean(raw_signal))
    median_shift = float(np.median(filtered_signal) - np.median(raw_signal))
    var_before = float(np.var(raw_signal))
    var_after  = float(np.var(filtered_signal))
    artifact_variance_reduction_pct = ((var_before - var_after)/var_before*100.0) if var_before != 0 else 0.0
    return dict(
        peak_before=peak_before,
        peak_after=peak_after,
        peak_reduction=peak_reduction,
        mean_shift=mean_shift,
        median_shift=median_shift,
        artifact_variance_reduction_pct=artifact_variance_reduction_pct
    )


In [ ]:

# Console formatter with thresholds & time prefix
def format_metrics_console_inline(
        snr_db, impact, snr_method="variance_ratio",
        bp_enabled=False, bp_low=1.0, bp_high=40.0,
        artifact_suppression_thresh=20.0,
        drift_thresh_uv=5.0,
        variance_smoothing_thresh=5.0,
        window_time_prefix=""):
    import numpy as np
    if np.isinf(snr_db):
        snr_text = "∞ dB (noise≈0)"
        ratio_text = "Signal ≫ noise"
        sig_pct_text = "≈100% signal power"
    else:
        lin = 10 ** (snr_db / 10.0)
        sig_frac = lin / (1.0 + lin)
        snr_text = f"{snr_db:.2f} dB"
        ratio_text = f"Signal ~{lin:.1f}× stronger than noise"
        sig_pct_text = f"≈{sig_frac*100:.0f}% signal power"

    peak_before = impact['peak_before']
    peak_after  = impact['peak_after']
    peak_drop   = impact['peak_reduction']
    peak_pct    = (peak_drop / peak_before * 100.0) if peak_before > 0 else 0.0
    mean_shift   = impact['mean_shift']
    median_shift = impact['median_shift']
    var_drop_pct = impact['artifact_variance_reduction_pct']

    peak_tag  = " | Artifact Suppression" if peak_pct >= artifact_suppression_thresh else ""
    drift_tag = " | Drift Correction"     if (abs(mean_shift) >= drift_thresh_uv or abs(median_shift) >= drift_thresh_uv) else ""
    var_tag   = " | Smoothing Effect"     if var_drop_pct >= variance_smoothing_thresh else ""

    bp_text = f"[BP={'ON' if bp_enabled else 'OFF'} {bp_low}-{bp_high}Hz]"

    line1 = (f"{window_time_prefix}"
             f"[SNR: {snr_text} | {ratio_text} | {sig_pct_text}]  "
             f"[Peak: {peak_before:.2f}→{peak_after:.2f} μV (↓{peak_drop:.2f} μV, {peak_pct:.0f}%){peak_tag}]  "
             f"[Variance ↓{var_drop_pct:.1f}%{var_tag}]  {bp_text}")
    line2 = (f"[Baseline Shift: mean {mean_shift:+.2f} μV | median {median_shift:+.2f} μV{drift_tag}]  "
             f"[SNR method: {snr_method}]")
    return line1 + "\n" + line2



## Full Script (reference)
The following cell is a **non‑executing** reference stub. Keep your real app in your `.py` file; copy the snippets above as needed.


In [ ]:

# --- FULL SCRIPT (reference only) ---
# See your main .py file for the live hardware + plotting loop.
# This notebook is intended for documentation, math, and copy-paste snippets.
